# Cosmic-Ray Rejection

This time we will learn how to reject, i.e., erase, the unwanted signals: Cosmic-rays (CR). CR is a loosely defined term in observational astronomy used to describe high energetic particles (or the resulting radiation) originated from extraterrestrial sources. On CCD image, CR appears as a dot, short streak, as shown in the following image (image credit [Tony Hallas](http://astronomy.com/magazine/tony-hallas/2010/07/tony-hallas-cosmic-imaging-getting-clean-images)):

![CRexample](http://astronomy.com/-/media/import/images/1/e/e/september-2010-cosmic-rays.jpg?mw=600)

